In [1]:
from transformers import BertTokenizer, BertModel
import torch
import pandas as pd
import csv
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from torch.utils import data
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics import classification_report
import tqdm

In [16]:
#Uncomment for Aspect Category Experiments
train = 'data/aspect_final_train.csv'
test = 'data/aspect_final_test.csv'

#Uncomment for Polarity Experiment
#train = 'data/polarity_final_train.csv'
#test = 'data/aspect_final_test.csv'

with open(train) as fp:
    reader = csv.reader(fp, delimiter=",", quotechar='"')
    train_data = [row for row in reader]
with open(test) as fp:
    reader = csv.reader(fp, delimiter=",", quotechar='"')
    test_data = [row for row in reader]
    
print("TRAIN SAMPLES: {}".format(len(train_data)))
print("TEST SAMPLES: {}".format(len(test_data)))

TRAIN SAMPLES: 573
TEST SAMPLES: 144


In [10]:
def get_aspect(index_string, full_text):
    indices = index_string.replace('[','').replace(']','').split(',')
    aspect = ''
    for index in indices:
        aspect+= full_text.split()[int(index)] + ' '
    return aspect.strip()

def get_pos(sent_list,aspect_list):
    first_pos = sent_list.index(aspect_list[0])
    final_pos = []
    for i in range(0,len(aspect_list)):
        final_pos.append(first_pos+i)
    return final_pos    

class TalkLitDataset(data.Dataset):
    def __init__(self, tagged_sents):
        sents, aspects, tags = [], [], [] # list of lists
        for sent in tagged_sents:
            sent_tokens = tokenizer.encode(sent[0])
            try:
                aspect_tokens = tokenizer.encode(get_aspect(sent[1],sent[0]))[1:-1]
            except:
                print(sent[1])
                print(sent[0])
                continue
            pos_aspects = get_pos(sent_tokens, aspect_tokens)
            tag = sent[2]
            sents.append(sent_tokens)
            aspects.append(pos_aspects)
            tags.append(tag)
            
        self.sents, self.aspects, self.tags = sents, aspects, tags

    def __len__(self):
        return len(self.sents)

    def __getitem__(self, idx):
        words, aspects, tags = self.sents[idx], self.aspects[idx], tag2idx[self.tags[idx]] # words, tags: string list
        return words, aspects, tags

In [11]:
tags = []
for dat in train_data:
    tags.append(dat[2])
    
tags = list(set(tags))
",".join(tags)
tag2idx = {tag:idx for idx, tag in enumerate(tags)}
idx2tag = {idx:tag for idx, tag in enumerate(tags)}
print(tag2idx)

{'META': 0, 'JURY': 1, 'ALLO-REFERENCES': 2, 'CONTENDER': 3, 'READING': 4, 'None': 5, 'TEXT': 6, 'ONSITE-AUDIENCE': 7}


In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [6]:
tokenizer = BertTokenizer.from_pretrained("bert-base-german-cased")
model = BertModel.from_pretrained("bert-base-german-cased")

In [7]:
class Net(nn.Module):
    def __init__(self, vocab_size=None):
        super().__init__()
        self.model = BertModel.from_pretrained("bert-base-german-cased", output_hidden_states=True)
        self.fc = nn.Linear(768, vocab_size)
        self.device = device

    def forward(self, sent, aspects, y):
        sent = torch.LongTensor(sent).to(device)
        y = torch.LongTensor(y).to(device)
        input_ids = sent.unsqueeze(0)  # Batch size 1
        with torch.no_grad():
            outputs = self.model(input_ids)
            last_hidden_states = outputs.last_hidden_state[0]  # The last hidden-state is the first element of the output tuple
            start = 0
            end = len(last_hidden_states)-1
            context_window = 5
            
            if aspects[0]-context_window>0:
                start = aspects[0]-context_window
            if aspects[-1]+context_window<len(last_hidden_states)-1:
                end = aspects[-1]+context_window
                
            all_aspects = []
            for i in range(start,end):
                all_aspects.append(i)
            bert_embeds = torch.zeros(len(all_aspects),768)
            for i, aspect in enumerate(all_aspects):
                bert_embeds[i] = last_hidden_states[aspect]
                
            bert_embeds = torch.mean(bert_embeds, axis=0)
        logits = self.fc(bert_embeds)
        y_hat = logits.argmax(-1)
        return logits, y, y_hat, bert_embeds

In [12]:
train_dataset = TalkLitDataset(train_data)
train_iter = data.DataLoader(dataset=train_dataset,
                             batch_size=1,
                             shuffle=True,
                             num_workers=0)

test_dataset = TalkLitDataset(test_data)
test_iter = data.DataLoader(dataset=test_dataset,
                             batch_size=1,
                             shuffle=True,
                             num_workers=0)



In [13]:
model = Net(vocab_size=len(tag2idx))
model.to(device)
model = nn.DataParallel(model)

X_train = []
y_train = []
X_test = []
y_test = []

for i, batch in tqdm.tqdm(enumerate(train_iter)):
        words, aspects, y = batch
        _y = y
        logits, y, _, bert_embeds = model(words, aspects, y)
        logits = logits.view(-1, logits.shape[-1])
        y = y.view(-1)
        bert_embeds = bert_embeds.cpu().numpy()
        y = int(y.cpu().numpy()[0])
        X_train.append(bert_embeds)
        y_train.append(y)
        
for i, batch in tqdm.tqdm(enumerate(test_iter)):
        words, aspects, y = batch
        _y = y
        logits, y, _, bert_embeds = model(words, aspects, y)
        logits = logits.view(-1, logits.shape[-1])
        y = y.view(-1)
        bert_embeds = bert_embeds.cpu().numpy()
        y = int(y.cpu().numpy()[0])
        X_test.append(bert_embeds)
        y_test.append(y)

573it [01:12,  7.89it/s]
144it [00:17,  8.14it/s]


In [26]:
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB

names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes"]


classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB()]

for name, clf in zip(names, classifiers):
    clf.fit(X_train, y_train)
    predicted = clf.predict(X_test)
    print("CLASSIFIER: {}".format(name))
    print(classification_report(y_test, predicted))

CLASSIFIER: Nearest Neighbors
              precision    recall  f1-score   support

           0       0.60      0.72      0.66        57
           1       0.33      0.40      0.36         5
           2       0.64      0.49      0.55        37
           4       1.00      0.33      0.50         3
           5       0.26      0.41      0.32        22
           6       0.00      0.00      0.00         8
           7       0.86      0.46      0.60        13

    accuracy                           0.53       145
   macro avg       0.53      0.40      0.43       145
weighted avg       0.55      0.53      0.52       145

CLASSIFIER: Linear SVM
              precision    recall  f1-score   support

           0       0.66      0.84      0.74        57
           1       0.00      0.00      0.00         5
           2       0.72      0.76      0.74        37
           4       0.67      0.67      0.67         3
           5       0.44      0.36      0.40        22
           6       0.00  

/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


CLASSIFIER: RBF SVM
              precision    recall  f1-score   support

           0       0.41      1.00      0.58        57
           1       0.00      0.00      0.00         5
           2       0.00      0.00      0.00        37
           4       1.00      0.33      0.50         3
           5       1.00      0.14      0.24        22
           6       0.00      0.00      0.00         8
           7       0.00      0.00      0.00        13

    accuracy                           0.42       145
   macro avg       0.34      0.21      0.19       145
weighted avg       0.33      0.42      0.28       145



/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/gaussian_process/ker

CLASSIFIER: Gaussian Process
              precision    recall  f1-score   support

           0       0.64      0.86      0.74        57
           1       0.00      0.00      0.00         5
           2       0.72      0.70      0.71        37
           4       0.67      0.67      0.67         3
           5       0.50      0.41      0.45        22
           6       0.00      0.00      0.00         8
           7       0.67      0.62      0.64        13

    accuracy                           0.65       145
   macro avg       0.46      0.46      0.46       145
weighted avg       0.59      0.65      0.61       145

CLASSIFIER: Decision Tree
              precision    recall  f1-score   support

           0       0.43      0.68      0.53        57
           1       0.00      0.00      0.00         5
           2       0.50      0.35      0.41        37
           4       0.50      0.33      0.40         3
           5       0.13      0.09      0.11        22
           6       0.00

...

/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


CLASSIFIER: Neural Net
              precision    recall  f1-score   support

           0       0.65      0.74      0.69        57
           1       0.50      0.20      0.29         5
           2       0.71      0.68      0.69        37
           4       0.50      0.67      0.57         3
           5       0.36      0.41      0.38        22
           6       0.00      0.00      0.00         8
           7       0.50      0.54      0.52        13

    accuracy                           0.59       145
   macro avg       0.46      0.46      0.45       145
weighted avg       0.56      0.59      0.57       145



/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


CLASSIFIER: AdaBoost
              precision    recall  f1-score   support

           0       0.61      0.47      0.53        57
           1       0.00      0.00      0.00         5
           2       0.32      0.81      0.46        37
           4       0.00      0.00      0.00         3
           5       0.14      0.05      0.07        22
           6       0.00      0.00      0.00         8
           7       0.00      0.00      0.00        13

    accuracy                           0.40       145
   macro avg       0.15      0.19      0.15       145
weighted avg       0.35      0.40      0.34       145

CLASSIFIER: Naive Bayes
              precision    recall  f1-score   support

           0       0.68      0.68      0.68        57
           1       1.00      0.40      0.57         5
           2       0.58      0.70      0.63        37
           4       0.75      1.00      0.86         3
           5       0.46      0.50      0.48        22
           6       0.00      0.00

/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
